In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(100)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header

import time

# = = = = = = = = = = = = = = =

import math

# = = = = = = = = = = = = = = =

with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/input.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global api, list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        url_parse = urlparse(crawler_series['Url'])

        dict_param = parse_qs(url_parse.query)

        data = {'getArticles': {'applyDqmRules': True,
                                'articleCountry': 'DE',
                                'articleStatusIds': [],
                                'criteriaFilters': [],
                                'dataSupplierIds': [int(dict_param['brands'][0].strip())],
                                'filterQueries': ['(dataSupplierId NOT IN (4978,4982))'],
                                'genericArticleIds': [int(dict_param['groups'][0].strip())],
                                'includeAccessoryArticles': False,
                                'includeAll': False,
                                'includeArticleCriteria': True,
                                'includeArticleLogisticsCriteria': True,
                                'includeArticleStatusFacets': True,
                                'includeArticleText': True,
                                'includeComparableNumbers': True,
                                'includeCriteriaFacets': True,
                                'includeDataSupplierFacets': True,
                                'includeGTINs': True,
                                'includeGenericArticleFacets': True,
                                'includeGenericArticles': True,
                                'includeImages': True,
                                'includeLinkages': True,
                                'includeLinks': True,
                                'includeMisc': True,
                                'includeOEMNumbers': True,
                                'includePDFs': True,
                                'includePartsListArticles': False,
                                'includePrices': True,
                                'includeReplacedByArticles': True,
                                'includeReplacesArticles': True,
                                'includeTradeNumbers': True,
                                'lang': 'qa',
                                'linkagesPerPage': 100,
                                'page': 1,
                                'perPage': 1_000,
                                'provider': 23365,
                                'sort': [{'field': 'articleNumber', 'direction': 'asc'}]}}

        count_retry = 0
        while True:
            count_retry += 1

            try:
                url_check = f'https://web.tecalliance.net/tecdoc/qa/parts/search?brands={dict_param['brands'][0].strip()}&groups={dict_param['groups'][0].strip()}'

                resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                     data=json.dumps(data),
                                     headers=get_header(**{'x-api-key': api}),
                                     timeout=(5, 15))

                if resp.status_code == 200:
                    dict_ = resp.json()

                    if dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                        print(f'fuck api {api}')
                        time.sleep(3)
                        with open('./api.txt', 'r') as api_txt:
                            api = api_txt.read()
                    elif dict_['status'] == 200:
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        count = dict_['totalMatchingArticles']

        # = = = = = = = = = = = = = = =

        if count <= 0:
            raise

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.append({'No': crawler_series['No'],
                                      'Url': url_check,
                                      'SKU Count': count,
                                      'Page': math.ceil(count / 1_000)})

        # = = = = = = = = = = = = = = =

        crawler_status = True
    except:
        crawler_series['Check_Url'] = url_check

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/1.page-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./file/page_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2

fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
fuck api 2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy4zdRrpNvX1cAV6SRPvP
[